In [1]:
import requests
from urllib.request import urlopen
from urllib.parse import urlencode,unquote,quote_plus
import urllib
import lxml
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
import time
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans

In [2]:
판례목록_df = pd.read_html("법령검색목록.html")
판례목록_df = 판례목록_df[0]
# 판례목록_df = 판례목록_df[(판례목록_df["판결유형"] == "민사") | (판례목록_df["판결유형"] == "일반행정") | (판례목록_df["판결유형"] == "가사") | (판례목록_df["판결유형"] == "세무")]
판례목록_df = 판례목록_df[(판례목록_df["판결유형"] == "민사")]
# for i in 판례목록_df["판례ID"]:
#     print("https://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID={}&type=HTML".format(i))
판례목록_df

,순번,판례ID,제목,법원,사건유형,판결유형,선고일자
3,4,216807,대여금,대법원,대법원,민사,2021.06.30
4,5,216803,손해배상(기),대법원,대법원,민사,2021.06.30
5,6,216801,손해배상(기),대법원,대법원,민사,2021.06.30
6,7,216811,수수료등반환청구,대법원,대법원,민사,2021.06.30
9,10,216809,저작권침해등,대법원,대법원,민사,2021.06.30
...,...,...,...,...,...,...,...
39991,39992,103548,판결경정,대법원,대법원,민사,1987.09.15
39992,39993,211021,판결경정,대법원,대법원,민사,1987.09.15
39994,39995,103542,"가옥명도,유익비청구",대법원,대법원,민사,1987.09.08
39996,39997,103535,건물철거,대법원,대법원,민사,1987.09.08


In [ ]:
# 판례 XML 상세조회
info = []
def crawling_func(url):
    global info
    text_판시사항 = ""
    text_판결요지 = ""
    text_참조조문 = ""
    text_참조판례 = ""
    text_주문 = ""
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'lxml-xml') #lxml-xml -> lxml 추가 설치 필요(Beautifulsoup과 별도로 설치)
        판시사항_soup = BeautifulSoup(soup.판시사항.get_text(),'lxml')
#         info.append(판시사항_soup.get_text())
        text_판시사항+=판시사항_soup.get_text()
        판결요지_soup = BeautifulSoup(soup.판결요지.get_text(),'lxml')
#         info.append(판결요지_soup.get_text())
        text_판결요지+=판결요지_soup.get_text()
        참조조문_soup = BeautifulSoup(soup.참조조문.get_text(),'lxml')
        for i in 참조조문_soup:
            try:
                text_참조조문+=i.get_text()
            except:
                pass
        참조판례_soup = BeautifulSoup(soup.참조판례.get_text(),'lxml')
        text_참조판례+=참조판례_soup.get_text()
        
        
        response = requests.get(url2)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'lxml-xml')
        try:
            i_list = soup.find("div",attrs={"class":"pgroup","id":"conScroll"}).find_all("p",attrs={"class":"pty4_dep1"})
            for i in i_list:
                if i.next_sibling.next_sibling.get_text()[1:-1]=="주 문":
                    text_주문+=i.next_sibling.next_sibling.next_sibling.next_sibling.get_text()
                    break
        except:
            pass
        info.append([ID,text_판시사항,text_판결요지,text_참조조문,text_참조판례,text_주문])
    except:
        print(url,"오류발생, 재귀호출 시작")
        time.sleep(1)
        crawling_func(url)
    finally:
        return

for idx,ID in enumerate(판례목록_df["판례ID"]):   
    #기본 url
    url = "http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID={}&type=XML".format(ID)
    url2 = "https://www.law.go.kr/precInfoP.do?mode=0&precSeq={}&vSct=*".format(ID)
    crawling_func(url)
    print(idx,ID)

# info

0 216807
1 216803
2 216801
3 216811
4 216809
5 216825
6 216831
7 217047
8 216835
9 216845
10 216843
11 216823
12 216833
13 216797
14 216847
15 216851
16 216849
17 216853
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=216867&type=XML 오류발생, 재귀호출 시작
18 216867
19 216659
20 216657
21 216683
22 216671
23 216673
24 216667
25 216663
26 216669
27 216661
28 216665
29 216655
30 215411
31 215415
32 215413
33 215423
34 215425
35 215421
36 216689
37 215135
38 215159
39 215155
40 215137
41 215139
42 215143
43 215161
44 215141
45 215157
46 215165
47 215133
48 214577
49 214583
50 214579
51 214601
52 214603
53 214573
54 214581
55 214599
56 214605
57 214207
58 214211
59 214215
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=214233&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=214233&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=214233&type=XML 오류발생, 재귀호출 시작
http://www

237 210787
238 210783
239 211457
240 211455
241 211459
242 210797
243 210793
244 210799
245 211465
246 211463
247 210795
248 210791
249 211013
250 210993
251 210803
252 210811
253 210813
254 210809
255 210819
256 210821
257 210551
258 210835
259 210837
260 210833
261 210565
262 210571
263 210827
264 210829
265 210569
266 210567
267 210547
268 210831
269 210855
270 210427
271 210859
272 210421
273 210869
274 210865
275 210861
276 210417
277 210857
278 210415
279 210423
280 210419
281 210867
282 210853
283 210871
284 210863
285 210413
286 210877
287 210899
288 210893
289 210895
290 210901
291 210897
292 209685
293 209687
294 209693
295 209599
296 209603
297 210763
298 209591
299 209587
300 209601
301 209597
302 209605
303 209122
304 209128
305 209124
306 209127
307 209125
308 209131
309 209130
310 209123
311 209134
312 209133
313 208801
314 208799
315 208798
316 208800
317 208795
318 208797
319 208790
320 208806
321 208787
322 208726
323 208732
324 208737
325 208735
326 208733
327 208734

http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작


http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작


http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작


...

http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작


http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작


http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작


http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작


http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=186115&type=XML 오류발생, 재귀호출 시작
